## Take 17 lands data and create database. Each table in the database corresponds to a 17lands file

In [1]:
import os, sys, re
import json
import numpy as np
import random
import torch 
import polars as pl
from polars import selectors as cs
import pyarrow.feather as feather
import pandas as pd
import gc
from collections import defaultdict

In [2]:
from sqlalchemy import create_engine

In [3]:
from transformers import AutoTokenizer, AutoModel

In [4]:
import os, pathlib, itertools

In [5]:
scryfall_path = r"C:\Users\breuh\OneDrive\proggy\python\MTG\roberta\scryfall_json\default-cards-20230731210608.json"
scryfall_path = os.path.normpath(scryfall_path)

scryfall_pruned_path = r"C:\Users\breuh\OneDrive\proggy\python\MTG\roberta\scryfall_json\scryfall_pruned.json"
scryfall_pruned_path = os.path.normpath(scryfall_pruned_path)

lands_folder = r"C:\Users\breuh\OneDrive\proggy\python\MTG\roberta\17Lands"
lands_folder = pathlib.Path(os.path.normpath(lands_folder))

lands_csv_fnames = [f for f in lands_folder.glob("*.csv") if "combo" not in f.name]

In [6]:

# Base project path
project_root = pathlib.Path(r"C:/Users/breuh/OneDrive/proggy/python/MTG/roberta")

# Define file paths
scryfall_folder = project_root / "scryfall_json"

oracle_pattern = re.compile(r"oracle-cards-(\d{8})\d{6}\.json")
oracle_paths = sorted(list(filter(lambda x: oracle_pattern.search(x.name), scryfall_folder.iterdir())), key=lambda x: x.name, reverse=True)

scryfall_path = oracle_paths[0]
scryfall_pruned_path = project_root / "scryfall_json" / "scryfall_pruned.json"

# Define folder path
lands_folder = project_root / "17Lands"

# Get CSV files excluding 'combo'
lands_csv_fnames = [f for f in lands_folder.glob("*.csv") if "combo" not in f.name]


In [7]:
# Pre-define the card-related column substrings
col_filters = cs.contains("opening_") | cs.contains("drawn_") | cs.contains("tutored_") | cs.contains("hand_") | cs.contains("won")
card_selectors = cs.contains("opening_") | cs.contains("drawn_") | cs.contains("tutored_") | cs.contains("hand_")
won_selector = pl.col("won")
lands_dataframes = []

lazy_frames = [pl.scan_csv(fname).select(col_filters).with_columns(pl.all().cast(pl.Int32)) for fname in lands_csv_fnames]

In [ ]:
test = lands_csv_fnames[3]
test_df = pl.read_csv(test)

: 

In [120]:
test


WindowsPath('C:/Users/breuh/OneDrive/proggy/python/MTG/roberta/17Lands/game_data_public.DFT.TradSealed.csv')

In [8]:

CARD_PREFIXES = ("opening_hand_", "drawn_", "tutored_")

def per_file_card_sums(path, all_cards=None):
    """Given a path to a 17Lands CSV file, return a Polars LazyFrame with one column per card"""
    
    # exact 'won' and only prefix columns; no bad `|` mixing
    lf = pl.scan_csv(path).select(cs.starts_with(CARD_PREFIXES), pl.col("won"))

    cols = lf.collect_schema().names()
    # Build card_name -> [column names] in ONE pass
    groups = defaultdict(list)
    for c in cols:
        if c == "won":
            continue
        for p in CARD_PREFIXES:
            if c.startswith(p):
                card = c[len(p):]        # robust even if card names contain underscores
                groups[card].append(c)
                break
    # Cast only the columns we’ll sum (0/1 flags -> small ints). Cast 'won' too.
    to_cast = tuple({name for lst in groups.values() for name in lst})
    cast_exprs = []
    if to_cast:
        cast_exprs.append(pl.col(list(to_cast)).cast(pl.UInt16))
    if "won" in cols:
        cast_exprs.append(pl.col("won").cast(pl.UInt8))
    if cast_exprs:
        lf = lf.with_columns(*cast_exprs)

    # Horizontal sums per card (sorted for stable column order)
    sum_exprs = [
        pl.sum_horizontal(pl.col(groups[card])).alias(card)
        for card in sorted(groups.keys())
    ]

    # Ensure consistent schema across files, if requested
    if all_cards is not None:
        missing = sorted(all_cards.difference(groups.keys()))
        if missing:
            sum_exprs += [pl.lit(0, dtype=pl.UInt16).alias(card) for card in missing]

    # Return per-card totals + exact 'won'
    out_exprs = sum_exprs + ([pl.col("won")] if "won" in cols else [])
    return lf.select(out_exprs)


In [9]:
lazyframes = (per_file_card_sums(path) for path in lands_csv_fnames)


In [10]:
lands_df = pl.collect_all(lazyframes)

In [12]:
set(map(lambda c: c.split()[0], [c for c in lands_df[0].columns if c.__contains__("_")]))

set()

In [13]:
import pickle as pickle

In [14]:
ito_path = r"C:\Users\breuh\OneDrive\proggy\python\MTG\roberta\17Lands"
ito_path = pathlib.Path(os.path.normpath(ito_path))
ito_path = ito_path / r"gamedata_itosize.pkl"

name_pattern = r"\.(\D*)\.(\D*).csv"
name_pattern = re.compile(name_pattern)

def get_file_code(path):
    set_, format_ = name_pattern.search(path).groups()
    return f"{set_}_{format_}"

In [15]:

with open(ito_path, "wb") as f:
    
    itosize = {get_file_code(name.name):len(df) for name, df in zip(lands_csv_fnames, lands_df)}
    pickle.dump(itosize, f)

In [17]:
import adbc_driver_sqlite.dbapi as sqlite_adbc
from pathlib import Path
import sqlite3

In [18]:

savename =  "training_database.db"
name_pattern = re.compile(name_pattern)


engine = create_engine(f"sqlite:///{savename}", future=True)

In [19]:
connection_uri = f"sqlite:///{savename}"

# --- Main Loop ---
for df, name in zip(lands_df, lands_csv_fnames):
    print(f"Processing {name_pattern.search(name.name).groups()}...")
    set_, format_ = name_pattern.search(name.name).groups()
    table_name = f"{set_}_{format_}"

    print(f"Writing table {table_name}, tablesize: {df.shape}...")

    # The direct Polars write - no more Pandas! 🚀
    # This single line replaces to_pandas() and to_sql().
    df.write_database(
        table_name=table_name,
        connection=connection_uri,
        if_table_exists="replace",
        engine="adbc"  # Specify the fast backend ('connectorx' or 'adbc'). adbc significantly faster for SQLite
    )

    print(f"Inserted {len(df)} rows into {table_name}")

    # Clean up memory
    del df
    gc.collect()

print("\nDatabase creation complete.")

Processing ('BRO', 'Sealed')...
Writing table BRO_Sealed, tablesize: (41364, 336)...
Inserted 41364 rows into BRO_Sealed
Processing ('BRO', 'TradSealed')...
Writing table BRO_TradSealed, tablesize: (9026, 336)...
Inserted 9026 rows into BRO_TradSealed
Processing ('DFT', 'Sealed')...
Writing table DFT_Sealed, tablesize: (45375, 287)...
Inserted 45375 rows into DFT_Sealed
Processing ('DFT', 'TradSealed')...
Writing table DFT_TradSealed, tablesize: (6906, 287)...
Inserted 6906 rows into DFT_TradSealed
Processing ('KHM', 'PremierDraft')...
Writing table KHM_PremierDraft, tablesize: (440826, 322)...
Inserted 440826 rows into KHM_PremierDraft
Processing ('KTK', 'Sealed')...
Writing table KTK_Sealed, tablesize: (15297, 255)...
Inserted 15297 rows into KTK_Sealed
Processing ('KTK', 'TradSealed')...
Writing table KTK_TradSealed, tablesize: (2558, 255)...
Inserted 2558 rows into KTK_TradSealed
Processing ('LCI', 'Sealed')...
Writing table LCI_Sealed, tablesize: (54609, 292)...
Inserted 54609 row

In [20]:
from sqlalchemy import MetaData, select, func, literal_column, column

engine = create_engine(f"sqlite:///{savename}", future=True)

metadata = MetaData()
metadata.reflect(bind=engine)

In [21]:

table = metadata.tables['LTR_Sealed']

In [157]:
from sqlalchemy import and_

In [ ]:
def generate_random_combo(table, N=2):


In [161]:
N = 2

# columns to *test* (> 0); exclude 'won' from the condition set
candidates = [c for c in table.c if c.key != "won"]

combo = random.sample(candidates, N)              # list[Column]
cols_to_select = [*combo, table.c.won]           # include 'won' in SELECT

query = select(*cols_to_select).where(and_(*[c > 0 for c in combo]))

with engine.connect() as conn:
    rows = conn.execute(query).all()

print(np.sum(rows, axis = 0)/ len(rows))

[1.         1.28571429 0.71428571]


In [162]:
rows

[(1, 2, 1), (1, 2, 1), (1, 1, 1), (1, 1, 0), (1, 1, 1), (1, 1, 1), (1, 1, 0)]

In [142]:
combo

[Column('Press the Enemy', INTEGER(), table=<LTR_Sealed>),
 Column('Forest', INTEGER(), table=<LTR_Sealed>)]

In [146]:
rows

[(1, 1, 1), (1, 1, 1), (1, 1, 0), (1, 1, 0), (1, 1, 1), (1, 1, 1), (1, 1, 1)]

In [50]:
with engine.connect() as conn:
    result = conn.execute(select(table.c['won']))
    print(result.all())

[(0,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (1,), (1,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (1,), (1,), (0,), (1,), (0,), (0,), (1,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (0,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (1,), (0,), (0,), (1,), (0,), (0,), (1,), (1,), (0,), (0,), (1,), (1,), (0,), (0,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (0,), (1,), (1,), (0,), (1,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (1,), (0,), (0,), (1,), (1,), (1,), (0,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (0,), (1,), (0,), (0,), (1,), (1,), (1,), (1,), (1,), (0,), (0,), (1,), (0,), (0,), (0,), (1,), (1,), (1,), (1,), (0,), (0,), (0,), (1,), (1,), (0,), (1,), (0,

In [33]:
with open(ito_path, "rb") as f:
    itosize = pickle.load(f)

table_size = itosize.get("LTR_Sealed", None)

In [40]:
query = select(
    func.min(literal_column("rowid")),
    func.max(literal_column("rowid"))
).select_from(table)

with engine.connect() as conn:
    result = conn.execute(query)

In [39]:
table_names = list(metadata.tables.keys())
def get_random_table(table_names=table_names):
    return np.random.choice(table_names)

def get_random_table(table_names=table_names):
    table_name = get_random_table(table_names)
    table = metadata.tables[table_name]
    return table

In [40]:
def get_combo_columns(table, combos = 2):
    card_columns = [col for col in table.c.keys() if col != "won"]
    selected_columns = random.sample(card_columns, combos)
    return selected_columns

In [41]:
combo = get_combo_columns(table, combos=2)
selections = [table.c[name] for name in combo] + [table.c["won"]]
with engine.connect() as conn:
    res = conn.execute(select(*selections).limit(5))
    res.fetchall()

In [ ]:



def get_random_row(table_names=table_names):
    table = get_random_table(table_names)
    card_names = get_combo_columns(table, combos=2)
    card_selections = [table.c[name] for name in card_names]
    with engine.connect() as conn:
        sub_table = select(*card_selections).subquery()
        N = itosize[table.name]
        r = random.randint(1, N)
        query = select(sub_table).where(literal_column("rowid") >= r).order_by(literal_column("rowid")).limit(1).fetchone()
        result = conn.execute(query)
        row = result.fetchone()
    return row

    

    N = itosize(table_name)
    randint = random.randint(1, N)
    query = select(table).where(literal_column("rowid") == randint).limit(1)
    with engine.connect() as conn:
        result = conn.execute(query)
        row = result.fetchone()
    return row

In [ ]:
get_random_table()

: 

1